#**TODO**

Refine keywords : strip \n, ", ' etc ...

Handle ill-encoded keywords : utf-8 ...?

#**Import Modules**

In [0]:
import openpyxl

In [0]:
!pip install xlsxwriter
import xlsxwriter

In [0]:
import pandas as pd

In [0]:
import numpy as np

In [0]:
import gc

In [0]:
from os import listdir
from os.path import isfile, join

In [0]:
from google.colab import files

#**Mount Google Drive**

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
cd drive/My Drive/Colab Notebooks/Paper/Trade

[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/Paper/Trade'
/content/drive/My Drive/Colab Notebooks/Paper/Trade


In [0]:
ls

Crawler/  Data/  Paper-Authors/  Paper-Keywords/


#**Read and Process Data**

In [0]:
def pair_to_vector(idx, length):
    res = [0] * length
    for i in idx:
        res[i] = 1
    return res

In [0]:
DATAPATH = "./Data/raw/International/raw.xlsx"

In [0]:
keywords = []
papers = []
keyword_paper_pair = []
curr_idx = -1

# open workbook
wb = openpyxl.load_workbook(DATAPATH)
ws = wb.active
MAXROW = ws.max_row
MAXCOL = ws.max_column

for r in ws.rows:
    # ignore header
    if(r[0].row == 1):
        continue
    # halt loop
    if(r[0].row > MAXROW):
        break

    # fetch paper
    if(isinstance(r[2].value, str)):
        papers.append(r[2].value)
        # fetch keywords
        if(isinstance(r[4].value, str)):
            curr_idx += 1
            idx = 4
            while isinstance(r[idx].value, str) and idx < 16:
                kwd = r[idx].value.strip().upper()
                if(kwd in keywords):
                    keyword_paper_pair[keywords.index(kwd)].append(curr_idx)
                else:
                    keywords.append(kwd)
                    keyword_paper_pair.append([curr_idx])
                idx += 1
        elif(isinstance(r[16].value, str)):
            continue
            '''
            curr_idx += 1
            idx = 16
            while isinstance(r[idx].value, str) and idx < 26:
                kwd = r[idx].value.strip().upper()
                if(kwd in keywords):
                    keyword_paper_pair[keywords.index(kwd)].append(curr_idx)
                else:
                    keywords.append(kwd)
                    keyword_paper_pair.append([curr_idx])
                idx += 1
            '''
        else:
            continue
    else:
        continue
    
# encode keyword-paper pair to vector
keyword_vector = []
for i in range(len(keyword_paper_pair)):
    keyword_vector.append(pair_to_vector(keyword_paper_pair[i], curr_idx + 1))

wb.close()

In [0]:
len(papers)

19338

In [0]:
len(keywords)

13271

In [0]:
len(keyword_vector)

13271

#**Write Paper-Keyword Matrix to Excel**

In [0]:
TARGETPATH = "./Data/mat/International/pieces/"

In [0]:
wb = xlsxwriter.Workbook(TARGETPATH + "mat" + "_index.xlsx")
ws = wb.add_worksheet()
print("Started writing titles...")
for i in range(len(papers)):
    ws.write(i + 1, 0, papers[i])
print("Finished writing titles")
wb.close()

no = 1
wb = xlsxwriter.Workbook(TARGETPATH + "mat_" + str(no) + ".xlsx")
ws = wb.add_worksheet()
print("Started writing keywords...")
for j in range(len(keyword_vector)):
    ws.write(0, j % 5000, keywords[j])
    for i in range(len(keyword_vector[j])):
        ws.write(i + 1, j % 5000, keyword_vector[j][i])
    if((j + 1) % 5000 == 0):
        no += 1
        wb.close()
        print("Changed workbook...")
        wb = xlsxwriter.Workbook(TARGETPATH + "mat_" + str(no) + ".xlsx")
        ws = wb.add_worksheet()
print("Finished writing keywords")
wb.close()

Start writing titles...
Finished writing titles
Start writing keywords...
Changed workbook...
Changed workbook...
Finished writing keywords


#**Download Files**

In [0]:
files.download(TARGETPATH)